In [1]:
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq langchain==0.0.233 --progress-bar off
!pip install unstructured --progress-bar off
!pip install chromadb --progress-bar off
!pip install --qqq sentence_transformers --progress-bar off

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=88f15cceb735b7ba84a4471430c45bae03e0520c6f79d719535151c3f6b15b67
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470935 sha256=9b5b86cdf50536901619ead65b68441706de513d66e6356f7d292d80b36d6810
  Stored in directory: /root/.cache/pip/wheels/ea/dd/74/01b3ec7256a0800b99384e9a0f7620e358afc3a51a59bf9b49
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35417 sha256=571a2343f7724976fc10d8655e6792424b266dc7f7817c2e5b1191d16a4c06ef
  Stored in directory: /root/.cache/pip/wheels/02/39/c0/9eb1f7a42b4b38f6f333b6314d4ed11c46f12a0f7b78194f0d
Successfully built python-docx python-pptx olefile
  Attempting uninstall: cr

In [2]:
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
    GenerationConfig,
    TextStreamer
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

Load the model

In [4]:
MODEL_NAME = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, load_in_4bit=True, device_map="auto"
)

model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

ValueError: ignored

In [5]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1,
  "transformers_version": "4.30.0"
}

Note that we're loading the model in 8-bit mode. This will reduce the memory footprint and speed up the inference. We're also using the device_map parameter to load the model on the GPU.

Small Inference

In [6]:
Question = ("Which programming languages are more suited for beginners? ")

# prompt = f"""
#  ## Instruction : {Question}
#  ## Response :
#  """.strip()

prompt = """
The following is a friendly conversation between a human and an AI. The AI is
talkative and provides lots of specific details from its context.

Current conversation:

Human: Who is Spiderman?
AI:
""".strip()

In [7]:
input_ids = tokenizer(prompt,return_tensors='pt').input_ids
input_ids = input_ids.to(model.device)
# with torch.inference_model(output = model.generate(input=input_ids, temperature = 0.7, max_new_tokens = 1000))
output = model.generate(input_ids, temperature=0.7, max_new_tokens=1000)

In [8]:
print(input_ids)

tensor([[    2,   133,   511,    16,    10,  5192,  1607,   227,    10,  1050,
             8,    41,  4687,     4,    20,  4687,    16, 50118, 26594,  3693,
             8,  1639,  3739,     9,  2167,  1254,    31,    63,  5377,     4,
         50118,  1437, 50118, 42124,  1607,    35, 50118,  1437, 50118, 33837,
            35,  3394,    16, 15749,   397,   116, 50118, 15238,    35]],
       device='cuda:0')


In [9]:
print(tokenizer.decode(output[0]))

</s>The following is a friendly conversation between a human and an AI. The AI is
talkative and provides lots of specific details from its context.
 
Current conversation:
 
Human: Who is Spiderman?
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Spiderman.
AI: Sp

In [10]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1,
  "transformers_version": "4.30.0"
}

Embed the documents

In [21]:
!pip install sentence_transformers
import sentence_transformers
# embeddings = HuggingFaceEmbeddings(
#     model_name = "intfloat/e5-base-v2"
# )
embeddings = HuggingFaceEmbeddings()
#     model_name = "facebook/opt-350m"
# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2"
# )
# embeddings = HuggingFaceEmbeddings(
#     model_name = "facebook/opt-350m"

# )

NotImplementedError: ignored

We'll use a custom configuration for the text generation:

In [ ]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

Setting the temperature to 0 to get deterministic results.  We'll also set the repetition_penalty to 1.7 to reduce the chance (but not completely remove the occurrences) of the model repeating itself.

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False)

Trying using the huggingface pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1
)

In [ ]:
llm=HuggingFacePipepine(pipeline=pipe)

In [ ]:
response= llm(prompt  )

Loading the data

In [ ]:
Path = "/content/pdfs"
loader = UnstructuredPDFLoader("/content/pdfs/bd.pdf")
loader

In [ ]:
documents = loader.load_and_split()
documents

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size= 1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [ ]:
db = Chroma.from_documents(texts,embeddings)

In [ ]:
# db.similarity_search()

##Conversational Chain

In [ ]:
template= """
  You are a product analyst chatbot that is taling to product managers. use only the chat history and the following information
  {context}
  to answer in a helpful manner to the question. if you don't know the answer - say that you don't know.
  keep your replies compassionate and informative
  {chat history}
  ## Input : {question}
  ## Response :
""".strip()

In [ ]:
prompt = PromptTemplate(input_variables=["context", "question", "chat_history"], template = template)

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="## Input",
    ai_prefix="## Response",
    output_key="answer",
    return_messages=True
)

In [ ]:
chain = ConversationalretrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt":prompt},
    return_source_documents=True,
    verbose=True
)

In [ ]:
question = "What are rhe features of Brand Protector?"
answer = chain(question)

##QA Chain with Memory

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="## Input",
    ai_prefix="## Response",
    input_key="question"
    output_key="output_text",
    return_messages=False
)

In [ ]:
chain = load_qa_chain(
    llm,
    chain_type="stuff",
    prompt=prompt,
    memory=memory,
    verbose=True
)

In [ ]:
question = "What are rhe features of Brand Protector?"
docs = db.similarity_search(question)
answer = chain({"input_documents":docs, "question":question})

##Support Chatbot

In [ ]:
DEFAULT_TEMPLATE = """
  ## Instruction : You are a product analyst chatbot that is taling to product managers. use only the chat history and the following information
  {context}
  to answer in a helpful manner to the question. if you don't know the answer - say that you don't know.
  keep your replies compassionate and informative
  {chat history}
  ## Input : {question}
  ## Response :
""".strip()

class Chatbot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        documents_dir:Path,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = True
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template=prompt_template,
        )
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(documents_dir, embeddings)

    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt= PromptTemplate,
        verbose: bool = True:
    ):
        memory = ConversationBufferMemory(
          memory_key="chat_history",
          human_prefix="## Input",
          ai_prefix="## Response",
          input_key="question"
          output_key="output_text",
          return_messages=False
        )

        return load_qa_chain(
            text_pipeline,
            chain_type = "stuff",
            prompt=prompt,
            memory=memory,
            verbose=verbose
        )

    def _embed_data(
        self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
    ) -> Chroma:
        loader = loader = UnstructuredPDFLoader("/content/pdfs/bd.pdf")
        documents = loader.load_and_split()
        text_splitter = CharacterTextSplitter(chunk_size= 1000, chunk_overlap=20)
        texts = text_splitter.split_documents(documents)
        return Chroma.from_documents(texts, embeddings)


    def __call__(self, user_input: str) -> str:
      docs = self,db,similarity_search(user_input)
      return self.chain.run({"input_documents": docs, "question": user_input})

In [ ]:
chatbot = Chatbot(llm, embeddings,"/content/pdfs/bd.pdf")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

while True:
  user_input = input("You: ")
  if user_input.lower() in ["bye", "goodbye"]:
      break
  answer = chatbot(user_input)
  print()